In [130]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import pandas as pd

In [131]:
Spark = pyspark.sql.SparkSession.builder.appName('apks').getOrCreate()

In [132]:
df = Spark.read.csv('Ecommerce-Customers.csv',inferSchema = True,header = True)

In [133]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler


In [134]:
df.describe

<bound method DataFrame.describe of DataFrame[Email: string, Address: string, Avatar: string, Avg Session Length: double, Time on App: double, Time on Website: double, Length of Membership: double, Yearly Amount Spent: double]>

In [135]:
assemler = VectorAssembler(inputCols = [
 
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'],outputCol= 'features')

In [136]:
output = assemler.transform(df)

In [137]:
output.select('features','Yearly Amount Spent').show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [138]:
final_data =output.select(['features','Yearly Amount Spent'])

In [139]:
train,test=final_data.randomSplit([0.7,0.3])

In [140]:
train.describe().show()


+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                337|
|   mean|  501.2763610530056|
| stddev|  77.09354257938152|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [141]:
test.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                163|
|   mean| 495.25696597811384|
| stddev|  83.81927212758927|
|    min|  275.9184206503857|
|    max|  765.5184619388373|
+-------+-------------------+



In [142]:
from pyspark.ml.regression import LinearRegression

In [143]:
op_lr = LinearRegression(labelCol='Yearly Amount Spent')

In [144]:
my_model = op_lr.fit(train)


In [145]:
finalmodel_result = my_model.evaluate(test)

In [146]:
finalmodel_result.rootMeanSquaredError

10.689960619258438

In [147]:
finalmodel_result.r2

0.9836342078905316

In [148]:
finalmodel_result.meanSquaredError


114.27525804129625

In [149]:
finalmodel_result.meanAbsoluteError

8.374700804075255

In [150]:
from pyspark.sql.functions import corr
df.select(corr('Yearly Amount Spent','Time on Website')).show()

+------------------------------------------+
|corr(Yearly Amount Spent, Time on Website)|
+------------------------------------------+
|                      -0.00264084467215...|
+------------------------------------------+

